In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import arange
import math
import datetime
import metodos_sistemas_lineares as sl

In [2]:
# Testando os métodos
A = [[4,-2,2],[-2,10,-7],[2,-7,30]]
B = [8,11,-31]
# A = [[1,2,-1],[2,3,-2],[1,-2,1]]
# B = [2,3,0]
# A = [[2,4,-2],[4,9,-3],[-2,-3,7]]
# B = [2,8,10]
print('Gauss: '+str(sl.Gauss(A,B)))
A = [[4,-2,2],[-2,10,-7],[2,-7,30]]
B = [8,11,-31]
print('LU: '+str(sl.LU(A,B)))
A = [[4,-2,2],[-2,10,-7],[2,-7,30]]
B = [8,11,-31]
print('Cholesky: '+str(sl.Cholesky(A,B)))
A = [[4,-2,2],[-2,10,-7],[2,-7,30]]
B = [8,11,-31]
print('Jacobi: '+str(sl.Jacobi(A,B,[0]*len(B),1000,10**-3)))
A = [[4,-2,2],[-2,10,-7],[2,-7,30]]
B = [8,11,-31]
print('Gauss-Seidel: '+str(sl.GaussSiedel(A,B,0,1000,10**-3)))

Gauss: (array([ 3.,  1., -1.]), 13)
LU: (array([ 3.,  1., -1.]), 5)
Cholesky: (array([ 3.,  1., -1.]), 4)
Jacobi: (array([ 2.99966454,  1.00026848, -1.00013994]), 195)
Gauss-Seidel: (array([ 3.00003139,  1.00000013, -1.00000206]), 91)


In [3]:
def c2(epsilon):
    return (math.exp(-1/math.sqrt(epsilon)) - 1)/(math.exp(1/math.sqrt(epsilon)) - math.exp(-1/math.sqrt(epsilon)))

def c1(epsilon,c2):
    return -1-c2(epsilon)

def analitica_PVC(x,epsilon,c1,c2):
    return c1(epsilon,c2) * math.exp(-x/math.sqrt(epsilon)) + c2(epsilon)* math.exp(x/math.sqrt(epsilon)) + 1

In [4]:
def discretizacao_PVC(h,epsilon):
    return ((h ** 2)/ epsilon)

In [ ]:
y_eps = []
y_analitica = []

tam = [1000,1500,2000]
it_thomas = []
it_jacobi = []
it_gauss = []
it_gs = []
it_lu = []
it_cholesky = []

for t in tam:
    print('Tam: '+str(t)+'\n')
    h = 1/t

    e = 0.1

    fx = []
    a = c = [-1]*len(np.arange(0,1,h))
    b = [2+(h**2)/e]*len(np.arange(0,1+h,h))
    y_analitica = [np.float32(analitica_PVC(x,e,c1,c2)) for x in arange(0,1+h, h)]

    # Adquirindo valores do termo dependente
    for i in arange(h,1,h):
        fx.append(np.float32(discretizacao_PVC(h, e)))
        

    M = sl.preenche_tridiagonal(a,b,c)

#     Adquirindo valores a partir do algoritmo de thomas
#     print('Analitica: '+str(y_analitica[1:-1]))


    inicio = datetime.datetime.now()
    y_disc, iteracoes = sl.Thomas(M,fx)
    it_thomas.append(iteracoes)
    fim = datetime.datetime.now()
    print('Thomas: Iteracoes: '+str(iteracoes))
    print('Duracao: '+str((fim-inicio).total_seconds()))
    
    print('\n')
    M = sl.preenche_tridiagonal(a,b,c)
    sl.critLinhas(M)
    print('\n')
    
    fx = []
    for i in arange(h,1,h):
        fx.append(np.float32(discretizacao_PVC(h, e)))

    M = sl.preenche_tridiagonal(a,b,c)
    inicio = datetime.datetime.now()
    x, iteracoes = sl.GaussSiedel(M,fx,0.5,t*10,10**-4)
    it_gs.append(iteracoes)
    fim = datetime.datetime.now()
    print('Gauss siedel: Iteracoes: '+str(iteracoes))
    print('Duracao: '+str((fim-inicio).total_seconds()))
    
    fx = []
    for i in arange(h,1,h):
        fx.append(np.float32(discretizacao_PVC(h, e)))

    M = sl.preenche_tridiagonal(a,b,c)
    
    inicio = datetime.datetime.now()
    x, iteracoes = sl.Jacobi(M,fx,[0.5]*len(fx),t*10,10**-4)
    it_jacobi.append(iteracoes)
    fim = datetime.datetime.now()
    print('Jacobi: Iteracoes: '+str(iteracoes))
    print('Duracao: '+str((fim-inicio).total_seconds()))
    fx = []

    for i in arange(h,1,h):
        fx.append(np.float32(discretizacao_PVC(h, e)))

    M = sl.preenche_tridiagonal(a,b,c)
    inicio = datetime.datetime.now()
    x, iteracoes = sl.Gauss(M,fx)
    it_gauss.append(iteracoes)
    fim = datetime.datetime.now()
    print('Gauss: Iteracoes: '+str(iteracoes))
    print('Duracao: '+str((fim-inicio).total_seconds()))

    

    fx = []
    for i in arange(h,1,h):
        fx.append(np.float32(discretizacao_PVC(h, e)))

    M = sl.preenche_tridiagonal(a,b,c)
    datetime.datetime.now()
    x, iteracoes = sl.LU(M,fx)
    it_lu.append(iteracoes)
    fim = datetime.datetime.now()
    print('LU: Iteracoes: '+str(iteracoes))
    print('Duracao: '+str((fim-inicio).total_seconds()))

    fx = []
    for i in arange(h,1,h):
        fx.append(np.float32(discretizacao_PVC(h, e)))

    M = sl.preenche_tridiagonal(a,b,c)
    inicio = datetime.datetime.now()
    x, iteracoes = sl.Cholesky(M,fx)
    it_cholesky.append(iteracoes)
    fim = datetime.datetime.now()
    print('Cholesky: Iteracoes: '+str(iteracoes))
    print('Duracao: '+str((fim-inicio).total_seconds()))


In [56]:
tam = [1000,1500,2000]

In [59]:
# Resultados obtidos após iterações
it_thomas = [1996,2996,3996] 
it_jacobi = [4689899760,10729010945,19187188010]
it_gauss = [332833499,1123875249,2664666999]
it_gs = [2356913910,5438607055,9753354105]
it_lu = [501500,1127250,2003000]
it_cholesky = [16616700,561375500,1331334000]

In [62]:
plt.plot(tam,it_jacobi,label='Jacobi')
plt.plot(tam,it_thomas,label='Thomas')
plt.plot(tam,it_gauss,label='Gauss')
plt.plot(tam,it_cholesky,label='Cholesky')
plt.plot(tam,it_lu,label='LU')
plt.plot(tam,it_gs,label='Gauss-Siedel')
plt.yscale('log')
plt.xlabel('Ordem da matriz')
plt.ylabel('Número de Iterações')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# plt.show()
plt.savefig('iteracoes',bbox_inches = 'tight')

In [10]:
def preencheEx2(n):
    A = np.array([[0.0]*n]*n)
    B = np.array([0.0]*n)
    
    for i in range(n):
        B[i] = 1/(i+n+1)
        for j in range(n):
            A[i][j] = 1/(i+j+1)
            
    return A,B

In [11]:
def norma_erro(A,B,x):
    n = len(B)
    m = 0
    for i in range(n):
        s = []
        for j in range(n):
            s.append(A[i][j]*x[i] - B[i])
        if m < max(s):
            m = max(s)
    return m

In [ ]:
tam = [10,100,1000]
erro_lu = []
erro_gauss = []
erro_gp = []
for t in tam:
    print(t)
    A,B = preencheEx2(t)
    x,it = sl.LU(A,B)
    erro_lu.append(norma_erro(A,B,x))
    
    A,B = preencheEx2(t)
    x,it = sl.Gauss(A,B)
    erro_gauss.append(norma_erro(A,B,x))

    A,B = preencheEx2(t)
    x,it = sl.Gauss_pivot(A,B)
    erro_gp.append(norma_erro(A,B,x))


10
100
1000


In [39]:
plt.plot(tam,erro_gauss,label='Gauss')
plt.plot(tam,erro_lu,label='LU')
plt.plot(tam,erro_gp,label='Gauss Pivot')

plt.yscale('log')
plt.xlabel('Ordem da matriz')
plt.ylabel('Erro Máximo')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# plt.show()
plt.savefig('erro.png',bbox_inches = 'tight')